# Web Scraper - Real State
Link: https://realpython.com/beautiful-soup-web-scraper-python/

Following the tutorial to get some web info in real-state in Argentina.

Webs I might be using to get the information:

* https://www.inmobusqueda.com.ar/casa-enbarrio-nunez-en-capital-federal.html
* https://www.remax.com.ar/
* https://www.topinmobiliario.com/home/index.ashx
* https://www.zonaprop.com.ar/
* https://www.2clics.com.ar/portales-inmobiliarios/

In [91]:
# Cotización del dolar
# Oficial site: https://estadisticasbcra.com/api/documentacion
# But..
#   Need to get a token by subscribing (don't want to do it for now)
#   Only oficial information, no dolar blue ..
import requests
import time

DURL = 'https://www.dolarsi.com/api/api.php?type=valoresprincipales'
resp = requests.get(DURL)

In [75]:
# import pprint
# pprint.PrettyPrinter(indent=4, depth=6).pprint(page.content)
# page.headers

for r in resp.json():
    casa = r['casa']
    if 'Blue' in casa['nombre']:
        print(f"compra dolar blue:{casa['compra']}")
        print(f"venta dolar blue:{casa['venta']}")
        # Taking 'venta' value of Dolar Blue
        usd_rate_change = float(casa['venta'].replace(',','.'))
        print(f"venta dolar blue:{usd_rate_change}")

compra dolar blue:127,000
venta dolar blue:132,000
venta dolar blue:132.0


In [125]:
#resp.json()[1]

print(f"Codificacion:{resp.apparent_encoding}")

for j in resp.json():
    print(j)
    print('---')

Codificacion:ascii
{'casa': {'compra': '72,390', 'venta': '77,390', 'agencia': '349', 'nombre': 'Dolar Oficial', 'variacion': '-0,040', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'compra': '127,000', 'venta': '132,000', 'agencia': '310', 'nombre': 'Dolar Blue', 'variacion': '-0,750', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'compra': 'No Cotiza', 'venta': '0', 'agencia': '311', 'nombre': 'Dolar Soja', 'variacion': '0', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'compra': '126,770', 'venta': '127,390', 'agencia': '312', 'nombre': 'Dolar Contado con Liqui', 'variacion': '-2,170', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'compra': '123,470', 'venta': '122,700', 'agencia': '313', 'nombre': 'Dolar Bolsa', 'variacion': '-1,460', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'compra': '9.852,070', 'venta': '12.350,000', 'agencia': '399', 'nombre': 'Bitcoin', 'variacion': '-0,27', 'ventaCero': 'TRUE', 'decimales': '3'}}
---
{'casa': {'nom

In [74]:
type(usd_rate_change)

float

In [17]:
resp.encoding

'iso-8859-1'

In [19]:
resp.headers

{'Date': 'Sun, 09 Aug 2020 13:51:44 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.6.39', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'Set-Cookie': 'usinr=2358967-b597eeed653f6ed94ae25a88acd64031e24630c3; expires=Mon, 10-Aug-2020 13:51:44 GMT; Max-Age=86400; path=/; domain=.inmobusqueda.com.ar, referer=deleted; expires=Thu, 01-Jan-1970 00:00:01 GMT; Max-Age=0; path=/; domain=.inmobusqueda.com.ar, pagina=rnuevos.casa-alquiler-enbarrio-nunez-en-capital-federal.html; expires=Mon, 10-Aug-2020 13:51:44 GMT; Max-Age=86400; path=/; domain=.inmobusqueda.com.ar, frase=-; expires=Mon, 10-Aug-2020 13:51:44 GMT; Max-Age=86400; path=/; domain=.inmobusqueda.com.ar, visita=5b0558f1b804e33ab2769c4ea13406f7; expires=Mon, 10-Aug-2020 13:51:44 GMT; Max-Age=86400; path=/; domain=.inmobusqueda.com.ar, testcookie=5b0558f1b804e33ab2769c4ea13406f7; expires=Mon, 10-Aug-2020 13:51:44 GMT; Max-Age=86400; path=/; domain=.inmobusqueda.com.ar', 'Connection': 'close', 'Transfer-Encoding': 'chunked', 'Co

## Using request + Beatuful Soup

### inmobusqueda

First step is to go the page with the desired filters and open it with `Developer Tools` from the browser.

Another way is to use wget from the CLI: `wget https://www.inmobusqueda.com.ar/casa-alquiler-enbarrio-nunez-en-capital-federal.html`. This will save the data in an html file so we can search there for some key words.

In [89]:
# import requests
from bs4 import BeautifulSoup

# Only for rent in Nuñez (2)
# URL = 'https://www.inmobusqueda.com.ar/casa-alquiler-enbarrio-nunez-en-capital-federal.html'
# Not filtering for rent or sell (10)
# URL = 'https://www.inmobusqueda.com.ar/casa-enbarrio-nunez-en-capital-federal.html'
# Not filtering by house type (appartament, house, PH) (131)
# Need to detect and handle pagination - This web shows 15 out of 131 adds...
# URL = 'https://www.inmobusqueda.com.ar/propiedades-enbarrio-nunez-en-capital-federal.html'
URL = 'https://www.inmobusqueda.com.ar/departamento-enbarrio-tribunales-en-capital-federal-90000-200000-dolares.html'
# URL = 'https://www.inmobusqueda.com.ar/propiedades-enbarrio-nunez-en-capital-federal-pagina-2.html'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [84]:
# soup.find(id='contenidoPropiedad251764') #.prettify()
a = '120.0000'
int(float(a))

129353*132

17074596

In [15]:
# Buscando en el Developer Tool de Chrome veo que esta clase me interesa
web_elems = soup.find_all(class_='letra2 cajaPremiumResultados ResultadoCaja')
# web_elems = soup.find_all(class_='derecha2019')    

In [100]:
count = 0
source = URL
header = ('address', 'offer_type', 'house_type', 'area', 'room_num', 'price_u$d', 'price_$', 
          'u$d_rate_change', 'source', 'description', 'last_update', 'add_id', 'time_stamp' )
print(header)
print('_.'*15)

dt_format = "%Y-%m-%d %H:%M:%S"
time_stamp = time.strftime(dt_format, time.gmtime(time.time()))

for elem in web_elems:
    # Each elem is a new BeautifulSoup object.
    # You can use the same methods on it as you did before.
    ficha = elem.find('div', class_='resultadoTipo').find('a').text.split(" en ")
    address = ficha[0]
    offer_type = ficha[1].strip() if len(ficha) > 1 else None
    # ----
    gral_desc = elem.find('div', class_='resultadoLocalidad').find('div').text
    house_type = gral_desc.split()[0]
    # ----
    features = elem.find('div', class_='resultadopublica').text.strip()
    ff = features.split()
    last_update = ff[-1]
    add_id = ff[-2]
    area = ff[ff.index('mts')-1]
    aux_garage_index = ff.index('Garage') if 'Garage' in ff else None
    garage = ff[aux_garage_index+1] if aux_garage_index else None
    
    if 'Monoamb' in ff:
        room_num = 1
    else:
        aux_amb_ii = ff.index('Dorm')
        if aux_amb_ii:
            room_num = int(ff[aux_amb_ii-1]) + 1
        else:
            room_num = None  
    # ----
    location_elem = elem.find('div', class_='resultadoPrecio')
    desc = elem.find('div', class_='resultadoDescripcion')

    if 'u$d' in location_elem.text:
        price_usd = float(location_elem.text.replace('u$d','').replace('.','')) 
        price_s = int(price_usd * usd_rate_change)
    
    row = (address, offer_type, house_type, area, room_num, price_usd, price_s, usd_rate_change,
          source, desc.text, last_update, add_id, time_stamp)
    
    # print('_'*30)
    print(row)
    print('_'*30)
    count += 1
        
print(f"Total adds:{count}")

('address', 'offer_type', 'house_type', 'area', 'room_num', 'price_u$d', 'price_$', 'u$d_rate_change', 'source', 'description', 'last_update', 'add_id', 'time_stamp')
_._._._._._._._._._._._._._._.
('tucuman al 1500  ', 'Venta', 'Departamento', '61.00', 2, 129000.0, 17028000, 132.0, 'https://www.inmobusqueda.com.ar/departamento-enbarrio-tribunales-en-capital-federal-90000-200000-dolares.html', 'Sup. Total 61m22 grandes ambientesque han sido convertidos en 3 ambientes.VENTA DIRECTA, el propietario no necesita comprar otra propiedad.Ubicado a 2 cuad...', '07-08-2020', 'IB-264149', '2020-08-17 21:31:16')
______________________________
('viamonte al 1300  ', 'Venta', 'Departamento', '47.00', 1, 130000.0, 17160000, 132.0, 'https://www.inmobusqueda.com.ar/departamento-enbarrio-tribunales-en-capital-federal-90000-200000-dolares.html', 'ALQUILER DEPARTAMENTO 1 AMBIENTE TRIBUNALES.Unidad de 47m2 al contrafrente, se encuentra ubicada en el corazón de Tribunales, a metros de Av. Urugua...', '07-0

In [96]:
count = 0
source = URL
header = ('address', 'offer_type', 'house_type', 'area', 'room_num', 'price_u$d', 'price_$', 
          'u$d_rate_change', 'source', 'description', 'last_update', 'add_id', 'time_stamp' )
print(header)

dt_format = "%Y-%m-%d %H:%M:%S"
time_stamp = time.strftime(dt_format, time.gmtime(time.time()))

for elem in web_elems:
    # Each elem is a new BeautifulSoup object.
    # You can use the same methods on it as you did before.
    ficha = elem.find('div', class_='resultadoTipo').find('a').text.split(" en ")
    address = ficha[0]
    offer_type = ficha[1].strip() if len(ficha) > 1 else None
    # ----
    gral_desc = elem.find('div', class_='resultadoLocalidad').find('div').text
    house_type = gral_desc.split()[0]
    # ----
    features = elem.find('div', class_='resultadopublica').text.strip()
    ff = features.split()
    last_update = ff[-1]
    add_id = ff[-2]
    area = ff[ff.index('mts')-1]
    aux_garage_index = ff.index('Garage') if 'Garage' in ff else None
    garage = ff[aux_garage_index+1] if aux_garage_index else None
    
    if 'Monoamb' in ff:
        room_num = 1
    else:
        aux_amb_ii = ff.index('Dorm')
        if aux_amb_ii:
            room_num = int(ff[aux_amb_ii-1]) + 1
        else:
            room_num = None
            
    # ----
    location_elem = elem.find('div', class_='resultadoPrecio')
    desc = elem.find('div', class_='resultadoDescripcion')
    #if None in (ficha, company_elem, location_elem, desc):
    #    continue
    # print(top)
    # print(f"FICHA:{ficha}")
    print(f"* Address: {address}")
    print(f"* Type   : {offer_type}")
    # print(gral_desc)
    print(f"* House Type   : {house_type}")
    # print("."*20)
    # print(features)
    # print(ff)
    # print("LENGHT:" + str(len(ff)))
    print(f"* Last update   : {last_update}")
    print(f"* Add id        : {add_id}")
    print(f"* Sup           : {area} m2")
    print(f"* Ambientes     : {room_num}")
    print(f"* Garage        : {garage}")
    # print("."*20)
    price_usd = float(location_elem.text.replace('u$d','').replace('.','')) if 'u$d' in location_elem.text else None
    price_s = int(price_usd * usd_rate_change)
    # print(f"* Precio        : {location_elem.text}")
    print(f"* Precio en U$D : {price_usd}")
    print(f"* Descripcion   : {desc.text}")    
    #break
    count += 1
    
    row = (address, offer_type, house_type, area, room_num, price_usd, price_s, usd_rate_change,
          source, desc.text, last_update, add_id, time_stamp)
    
    print('_'*30)
    print(row)
    print('_'*30)
    
    # print("-"*60)
    
print(f"Total adds:{count}")

#header = ('address', 'offer_type', 'house_type', 'area', 'room_num', 'price_u$d', 'price_$', 
#          'u$d_rate_change', 'source', 'description', 'last_update', 'add_id', 'time_stamp' )

('address', 'offer_type', 'house_type', 'area', 'room_num', 'price_u$d', 'price_$', 'u$d_rate_change', 'source', 'description', 'last_update', 'add_id', 'time_stamp')
* Address: tucuman al 1500  
* Type   : Venta
* House Type   : Departamento
* Last update   : 07-08-2020
* Add id        : IB-264149
* Sup           : 61.00 m2
* Ambientes     : 2
* Garage        : No
* Precio en U$D : 129000.0
* Descripcion   : Sup. Total 61m22 grandes ambientesque han sido convertidos en 3 ambientes.VENTA DIRECTA, el propietario no necesita comprar otra propiedad.Ubicado a 2 cuad...
______________________________
('tucuman al 1500  ', 'Venta', 'Departamento', '61.00', 2, 129000.0, 17028000, 132.0, 'https://www.inmobusqueda.com.ar/departamento-enbarrio-tribunales-en-capital-federal-90000-200000-dolares.html', 'Sup. Total 61m22 grandes ambientesque han sido convertidos en 3 ambientes.VENTA DIRECTA, el propietario no necesita comprar otra propiedad.Ubicado a 2 cuad...', '07-08-2020', 'IB-264149', '2020-08-

Now we can put all the information together in a csv to further process